In [1]:
import pandas as pd


data = pd.read_pickle('./reviews_processed.pkl')


In [3]:
dataTransposed = data.transpose()

dataTransposed.head()

,count,name,price,pricePer,rating,reviews,url,processedReviews
B079YF1K1B,120,"PhosphatidylSerine 300mg Per Serving, Made in ...",19.95,0.17,4.5,"[{'user': 'Beth and Dan Richard', 'rating': 1....",/PhosphatidylSerine-Phosphatidyl-Double-Wood-S...,"[[work, for, now, food, and, bought, three, di..."
B08XYDDXDC,120,"Nutricost Phosphatidylserine 400mg, 120 Capsul...",17.95,0.15,4.6,"[{'user': 'Barbara Greenan', 'rating': 1.0, 't...",/Nutricost-Phosphatidylserine-400mg-120-Capsul...,"[[to, improve, memory], [good, product]]"
B08LF5G6J1,90,Ultra High Purity Phosphatidylserine from Ital...,19.97,0.22,4.6,"[{'user': 'Michael K', 'rating': 5.0, 'text': ...",/Ultra-High-Purity-Phosphatidylserine-Italy-St...,"[[partner, and, are, on, our, second, bottle, ..."
B0013OXFTU,120,"NOW Supplemnets, Phosphatidyl Serine 100 mg wi...",31.95,0.27,4.6,"[{'user': 'dSavannah', 'rating': 5.0, 'text': ...",/NOW-Phosphatidyl-Serine-100mg-Capsules/dp/B00...,"[[take, and, have, tried, variety, of, supplem..."
B0013OXA90,60,Jarrow Formulas PS 100-60 Softgels - 100 mg Ph...,21.57,0.36,4.6,"[{'user': 'kswen', 'rating': 5.0, 'text': 'I'm...",/Jarrow-Formulas-Supports-Cognitive-Function/d...,"[[so, impressed, with, this, supplement, it, h..."


In [4]:
processedReviewsDF = dataTransposed['processedReviews']

arr = dataTransposed['processedReviews'].tolist()

arr_flat = []
for e1 in arr:
    if e1:
        for e2 in e1:
            if e2:
                arr_flat.append(e2)


In [5]:
import gensim


dictionary = gensim.corpora.Dictionary(arr_flat)

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break


/Users/CSUser/Documents/NLP/Project/Project/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


0 all
1 amazon
2 and
3 are
4 at
5 be
6 been
7 below
8 between
9 blocked
10 bottle


In [7]:
# Filter Tokens

dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=1000)

In [8]:
# Create bag of words

bow_corpus = [dictionary.doc2bow(doc) for doc in arr_flat]

# Check to see preview of Bag of Words

bow_doc_20 = bow_corpus[20]

for i in range(len(bow_doc_20)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_20[i][0], 
                                               dictionary[bow_doc_20[i][0]], 
bow_doc_20[i][1]))

Word 2 ("and") appears 5 time.
Word 14 ("have") appears 1 time.
Word 17 ("lot") appears 1 time.
Word 21 ("now") appears 1 time.
Word 22 ("of") appears 1 time.
Word 33 ("this") appears 1 time.
Word 34 ("three") appears 1 time.
Word 35 ("to") appears 2 time.
Word 36 ("two") appears 1 time.
Word 41 ("again") appears 1 time.
Word 42 ("also") appears 1 time.
Word 45 ("but") appears 1 time.
Word 46 ("can") appears 1 time.
Word 51 ("day") appears 1 time.
Word 65 ("my") appears 1 time.
Word 70 ("so") appears 1 time.
Word 73 ("take") appears 1 time.
Word 97 ("other") appears 1 time.
Word 103 ("supplement") appears 1 time.
Word 128 ("good") appears 1 time.
Word 139 ("noticed") appears 1 time.
Word 146 ("taking") appears 2 time.
Word 166 ("when") appears 1 time.
Word 175 ("went") appears 1 time.
Word 181 ("big") appears 1 time.
Word 193 ("medication") appears 1 time.
Word 205 ("thing") appears 1 time.
Word 268 ("started") appears 1 time.
Word 369 ("think") appears 1 time.
Word 402 ("difference") 

In [9]:
#Create tf-idf model on bow_corpus

from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]


In [10]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

/Users/CSUser/Documents/NLP/Project/Project/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/Users/CSUser/Documents/NLP/Project/Project/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [11]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.069*"good" + 0.046*"product" + 0.043*"no" + 0.041*"taste" + 0.036*"to" + 0.033*"very" + 0.032*"and" + 0.031*"fishy" + 0.028*"easy" + 0.024*"quality"
Topic: 1 
Words: 0.054*"and" + 0.038*"me" + 0.034*"my" + 0.027*"ha" + 0.024*"with" + 0.022*"helped" + 0.021*"more" + 0.020*"to" + 0.020*"work" + 0.019*"better"
Topic: 2 
Words: 0.066*"and" + 0.035*"this" + 0.031*"my" + 0.030*"it" + 0.029*"to" + 0.029*"have" + 0.018*"product" + 0.017*"feel" + 0.016*"of" + 0.016*"in"
Topic: 3 
Words: 0.042*"and" + 0.039*"to" + 0.035*"the" + 0.029*"it" + 0.023*"my" + 0.022*"wa" + 0.020*"this" + 0.016*"of" + 0.012*"in" + 0.012*"for"
Topic: 4 
Words: 0.057*"the" + 0.037*"it" + 0.031*"to" + 0.029*"and" + 0.027*"is" + 0.023*"this" + 0.022*"of" + 0.019*"that" + 0.016*"for" + 0.014*"product"
Topic: 5 
Words: 0.047*"for" + 0.039*"my" + 0.039*"and" + 0.026*"been" + 0.026*"this" + 0.024*"the" + 0.022*"it" + 0.020*"to" + 0.019*"have" + 0.018*"taking"
Topic: 6 
Words: 0.032*"to" + 0.027*"and" + 0.026*

In [12]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

/Users/CSUser/Documents/NLP/Project/Project/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/Users/CSUser/Documents/NLP/Project/Project/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/Users/CSUser/Documents/NLP/Project/Project/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein packa

Topic: 0 Word: 0.013*"fish" + 0.013*"quality" + 0.013*"oil" + 0.012*"omega" + 0.011*"the" + 0.010*"this" + 0.010*"it" + 0.010*"and" + 0.010*"product" + 0.010*"to"
Topic: 1 Word: 0.017*"love" + 0.012*"it" + 0.011*"this" + 0.011*"for" + 0.010*"have" + 0.010*"my" + 0.009*"product" + 0.009*"the" + 0.009*"and" + 0.009*"been"
Topic: 2 Word: 0.013*"work" + 0.013*"it" + 0.012*"the" + 0.011*"to" + 0.010*"dry" + 0.009*"and" + 0.008*"this" + 0.008*"take" + 0.008*"me" + 0.008*"my"
Topic: 3 Word: 0.017*"taste" + 0.013*"no" + 0.012*"fish" + 0.012*"very" + 0.011*"the" + 0.011*"lemon" + 0.011*"easy" + 0.010*"with" + 0.010*"to" + 0.010*"swallow"
Topic: 4 Word: 0.021*"burp" + 0.015*"fish" + 0.013*"no" + 0.011*"it" + 0.011*"the" + 0.010*"oil" + 0.009*"and" + 0.009*"this" + 0.009*"is" + 0.008*"fishy"
Topic: 5 Word: 0.031*"excellent" + 0.022*"eye" + 0.015*"product" + 0.012*"for" + 0.011*"the" + 0.011*"my" + 0.011*"it" + 0.011*"this" + 0.010*"great" + 0.010*"like"
Topic: 6 Word: 0.011*"the" + 0.010*"to" + 0

In [15]:
for index, score in sorted(lda_model[bow_corpus[4]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))



Score: 0.6364889740943909	 
Topic: 0.041*"to" + 0.038*"and" + 0.037*"my" + 0.027*"it" + 0.023*"for" + 0.021*"memory" + 0.021*"this" + 0.020*"the" + 0.017*"in" + 0.016*"have"

Score: 0.3252415060997009	 
Topic: 0.042*"and" + 0.039*"to" + 0.035*"the" + 0.029*"it" + 0.023*"my" + 0.022*"wa" + 0.020*"this" + 0.016*"of" + 0.012*"in" + 0.012*"for"

Score: 0.029832422733306885	 
Topic: 0.054*"and" + 0.038*"me" + 0.034*"my" + 0.027*"ha" + 0.024*"with" + 0.022*"helped" + 0.021*"more" + 0.020*"to" + 0.020*"work" + 0.019*"better"
